In [22]:
import os
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

In [23]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-07-08 03:18:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... failed: Connection timed out.
Retrying.

--2020-07-08 03:19:06--  (try: 2)  http://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... ^C


In [3]:
!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

--2020-07-08 02:37:00--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz [following]
--2020-07-08 02:37:01--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz’

download.php?f=Open 100%[===================>]   3.41G  17.8MB/s    in 3m 25s  

2020-07-08 02:40:27 (17.1 MB/s) - ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz

In [4]:
os.listdir('/tmp')
#os.mkdir('/tmp/word_embeddings')
os.mkdir('/tmp/lines')

In [5]:
!unzip glove.6B.zip -d /tmp/word_embeddings
os.listdir('/tmp/word_embeddings')

unzip:  cannot find or open glove.6B.zip, glove.6B.zip.zip or glove.6B.zip.ZIP.


FileNotFoundError: ignored

In [7]:
!gunzip -k download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz
!split -a 3 -l 100000 download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en /tmp/lines/lines-

In [ ]:
with open('/tmp/word_embeddings/glove.6B.100d.txt',"r") as f:
  word_to_vec_map = {}
  for line in f:
    line = line.strip().split()
    new_word = line[0]
    word_to_vec_map[new_word] = np.asarray(line[1:],dtype = 'float64')
print('Found %s word vectors'%len(word_to_vec_map))
print(word_to_vec_map["fuck"])

In [8]:
def filter_line(line):
  line = line.lower()
  line.strip()
  line = line.replace("\n"," ")
  line = line.replace(",","")
  line = line.replace(".","")
  line = line.replace("?","")
  line = line.replace("<","")
  line = line.replace(">","")
  line = line.replace("/","")
  line = line.replace(";","")
  line = line.replace(":","")
  line = line.replace("'","")
  line = line.replace("-"," ")
  line = line.replace("+","")
  line = line.replace("=","")
  line = line.replace("!","")
  line = line.replace("  "," ")
  line = "<startseq> " + line + "<endseq>"
  return line

In [9]:
dataset = []
for files in os.listdir('/tmp/lines'):
  path = os.path.join('/tmp/lines',files)
  f = open(path)
  i=0
  for line in f:
    line = filter_line(line)
    if i==100000:
      break
    dataset.append(line)
    i =i+1
  if i==100000:
    break

In [24]:
print(dataset[50001])

<startseq> but <endseq>


In [11]:
oov_token = "<OOV>"
trunc_type = 'post'
pad_type = 'post'
max_len = 20

In [12]:
tokenizer = Tokenizer(oov_token = oov_token)
tokenizer.fit_on_texts(dataset)
word_index = tokenizer.word_index
sequence = tokenizer.texts_to_sequences(dataset)
padded_sequence = pad_sequences(sequence, maxlen=max_len,padding = pad_type, truncating = trunc_type)
vocab_size = len(word_index)+1
print(vocab_size)

12910


In [13]:
index_to_word_map = {}
for word,index in word_index.items():
  index_to_word_map[index] = word

In [14]:
input = []
output = []
for j in range(0,99999,2):
  input_temp=padded_sequence[j]
  output_temp=padded_sequence[j+1]
  input.append(input_temp)
  output.append(output_temp)

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector 

In [21]:
for word, i in word_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector 
embedding_layer = Embedding(vocab_size,50,weights=[embedding_matrix],input_length=20,trainable=False)

NameError: ignored

In [27]:
batch_size = 40
embedding_dim = 100
print(len(input))
print(len(output))
max_length = 20

50000
50000


In [28]:
Decoder_LSTM = LSTM(256, return_state = True)
final_layer = Dense(vocab_size,activation = 'softmax')
repeator = RepeatVector(max_length)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('tanh', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [29]:
def attention(a,s_1):
  s_1 = repeator(s_1)
  concat = concatenator([a, s_prev])
  ie = densor1(concat)
  energy = densor2(ie)
  alpha = activator(energy)
  context = dotor([alpha,a])

  return context

In [30]:
def model():
  T_x = 20
  T_y = 20
  na = 128
  ns = 256
  s0 = np.zeros((10000, n_s))
  c0 = np.zeros((10000, n_s))

  reply = []

  glove_vector = embedding_layer(X)
  a = Bidirectional(LSTM(units = na,return_sequences=True))(glove_vector)

  s = s0
  c = c0
  for i in range (T_y):
    context = attention(a,s0)
    s0,_,c = Decoder_LSTM(input= context,intitial_state = [s,c])
    word = final_layer(s)
    reply.append(word)

    Model = model(inputs = [X],output = reply)

    return Model



In [31]:
model = model()
model.summary()

NameError: ignored

In [ ]:
opt = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999,  decay = 0.001)
model.compile(opt, loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])

outputlabels = list(outputlabels.swapaxes(0,1)
model.fit([input], output_labels, batch_size = 100, epochs = 5)

In [ ]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(5000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [ ]:
you = []
you_sequence = tokenizer.texts_to_sequences(you)
padded = pad_sequences(you_sequence,maxlen=20, truncating = 'post',padding = 'post')
reply = model.predict([padded])
reply = beam_search(reply)
print(reply)